In [1]:
# targetBoardId 추가 - DB 직접 연동 버전
# type 컬럼으로 게시판 선택
import json
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "crawling"

# ==================== 설정 영역 ====================
# 게시판 타입 (필터링용)
BOARD_TYPE = "event"  # ✨ 추가: hredu, cp, sales 등

# targetBoardId 값 설정
TARGET_BOARD_ID = "1433257621279768576"  # 사우회

# ======================================================================

def process_db(board_type, target_board_id):
    """DB에서 특정 type의 데이터만 읽고 targetBoardId 추가 후 DB에 업데이트"""
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 게시판 타입: {board_type}")  # ✨ 추가
    print(f"📂 targetBoardId: {target_board_id}\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 type의 데이터만 읽기
            query = "SELECT sourceId FROM board_posts WHERE type = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (board_type,))
            posts = cursor.fetchall()
            
            total_count = len(posts)
            print(f"✅ 총 {total_count}개의 게시글 발견\n")
            
            # 2. 특정 type의 게시글만 targetBoardId 업데이트
            update_query = """
                UPDATE board_posts 
                SET targetBoardId = %s
                WHERE type = %s
            """  # ✨ WHERE 조건 추가
            cursor.execute(update_query, (target_board_id, board_type))
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"완료! {total_count}개의 게시글 업데이트됨")
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 게시판 타입: {board_type}")  # ✨ 추가
            print(f"   - targetBoardId: {target_board_id}")
            print(f"   - 업데이트된 게시글: {total_count}개")
            print(f"   - DB 테이블: board_posts (targetBoardId 컬럼 업데이트)")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")

# 실행
if __name__ == "__main__":
    process_db(BOARD_TYPE, TARGET_BOARD_ID)  # ✨ board_type 인자 추가

📂 DB 연결 중...
📂 게시판 타입: event
📂 targetBoardId: 1433257621279768576

✅ 총 309개의 게시글 발견

완료! 309개의 게시글 업데이트됨

✅ 처리 완료!
   - 게시판 타입: event
   - targetBoardId: 1433257621279768576
   - 업데이트된 게시글: 309개
   - DB 테이블: board_posts (targetBoardId 컬럼 업데이트)

✅ DB 연결 종료
